## 회귀모형 성능평가

  - **문제 제기**: 단순히 모델을 만드는 것을 넘어, 예측 모델의 품질을 객관적인 숫자로 평가하고 여러 모델의 성능을 공정하게 비교하는 것은 매우 중요함. (예: 예측 가격과 실제 가격의 평균 오차가 100만 원인 모델은 좋은 모델인가?)
  - **학습 목표**: 회귀 모델의 '성적표'에 해당하는 다양한 \*\*성능 평가 지표(Performance Metrics)\*\*에 대해 배우고, Scikit-learn을 활용해 이를 직접 계산하고 해석하는 방법을 확인함.

## 핵심 원리 파헤치기 (Deep Dive)

1.  **예측모형의 `score` 메서드**: 모델 객체에 내장된 가장 기본적인 평가 방법. 회귀 모델의 경우 \*\*결정계수(R²)\*\*를 반환하여 모델의 설명력을 빠르게 확인할 때 사용함.
2.  **`sklearn.metrics` 모듈의 함수들**: **평균 제곱 오차(MSE), 평균 절대 오차(MAE)** 등 다양한 평가 지표를 계산하는 함수들을 제공하는 '성능 평가 도구 상자'. 모델 성능을 다각도로 분석할 수 있음.
3.  **`scoring` 매개변수**: `cross_val_score`나 `GridSearchCV`와 같이 교차 검증을 자동 수행하는 도구와 함께 사용됨. 원하는 평가 지표를 기준으로 모델 성능을 측정하도록 지정함.

### K-폴드 교차 검증: 모델 성능의 신뢰도를 높이는 방법

  - **문제 상황**: 데이터 양이 적을 때, 단 한 번의 훈련/테스트 데이터 분할은 운에 따라 성능 평가 결과가 크게 좌우될 수 있음.

  - **해결책**: **K-폴드(K-fold) 교차 검증**은 데이터를 K개의 부분(fold)으로 나누고, 각 부분을 한 번씩 검증용으로 사용하여 총 K번의 평가를 수행한 뒤 그 결과를 평균내는 방식임.

  - **장점**: 한 번의 평가보다 훨씬 더 안정적이고 신뢰할 수 있는 모델 성능 지표를 얻을 수 있음.

  ```
  sklearn.model_selection.KFold(n_splits=5, shuffle=False, random_state=None)
  ```

  - **`n_splits`**: 데이터를 나눌 개수(폴드 수).
      - **`shuffle`**: 데이터를 분할하기 전에 섞을지 여부.
      - **`random_state`**: `shuffle=True`일 때 사용되는 난수 시드로, 실험의 재현성을 보장함.


## 실전 코드 분석 및 적용 (Code Walkthrough & Application)

### 사이킷런에서 사용 가능한 평가 지표 확인하기

  - `sklearn.metrics` 모듈의 `SCORERS` 객체를 통해 `scoring` 매개변수로 사용할 수 있는 모든 평가 지표 목록을 확인할 수 있음.


In [1]:
# Scikit-learn에서 제공하는 모든 평가 지표의 키를 확인합니다.
from sklearn.metrics import get_scorer_names

# 사용 가능한 scoring 매개변수 이름들을 출력
print(get_scorer_names())

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'd2_absolute_error_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'matthews_corrcoef', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_max_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_negative_likelihood_ratio', 'neg_root_mean_squared_error', 'neg_root_mean_squared_log_error', 'normalized_mutual_info_score', 'positive_likelihood_ratio', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'rand_score', 'recall', 'recall_macro', 'recall_micro', 'recall_samples'

In [ ]:
import sklearn

    # estimator 값 지정을 하지 않아서 오류가 남
    # 여기서는 함수 구문만 확인
    # sklearn.model_selection.cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv=None, n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score='raise-deprecating')

##### 코드 해설

### 회귀모형 평가를 위한 `scoring` 속성

| scoring 속성의 값 | metrics 모듈의 회귀모형 평가함수 | 참고 |
| :--- | :--- | :--- |
| explained\_variance | metrics.explained\_variance\_score | 설명 분산 |
| neg\_mean\_absolute\_error | metrics.mean\_absolute\_error | (음수) MAE |
| neg\_mean\_squared\_error | metrics.mean\_squared\_error | (음수) MSE |
| neg\_mean\_squared\_log\_error | metrics.mean\_squared\_log\_error | (음수) MSLE |
| neg\_median\_absolute\_error | metrics.median\_absolute\_error | (음수) MedAE |
| r2 | metrics.r2\_score | 결정계수 |

  - **참고**: `scoring` 값에 `neg_` 접두사가 붙는 이유는 Scikit-learn의 최적화 로직이 점수를 '높이는' 방향으로 통일되었기 때문임. 오차(Error)는 낮을수록 좋으므로, 음수로 변환하여 값이 클수록(0에 가까울수록) 더 좋은 성능임을 나타냄.

##### 코드 해설

### `sklearn.metrics` 함수를 이용한 모델 평가

  - `sklearn.metrics` 모듈의 함수를 직접 사용하여 모델 성능을 측정하는 예제.

  - **주요 회귀모형 평가 함수**

| 함수 | 설명 |
| :--- | :--- |
| explained\_variance\_score | 설명된 분산 회귀 점수를 계산함. |
| mean\_absolute\_error | **평균 절대 오차(MAE)**. 실제값과 예측값 차이의 절댓값에 대한 평균. |
| mean\_squared\_error | **평균 제곱 오차(MSE)**. 실제값과 예측값 차이를 제곱하여 평균 낸 값으로, 큰 오차에 더 큰 페널티를 부여함. |
| mean\_squared\_log\_error | 평균 제곱 로그 오차(MSLE). 값의 스케일이 클 때 유용함. |
| median\_absolute\_error | 중앙값 절대 오차(MedAE). 이상치(outlier)에 덜 민감함. |
| r2\_score | **결정계수(R²)**. 모델이 데이터의 분산을 얼마나 잘 설명하는지 나타내며, 1에 가까울수록 좋음. |

In [2]:
# 필요한 라이브러리와 데이터를 불러옵니다.
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score

# 당뇨병 데이터셋 로드
diabetes = datasets.load_diabetes()
X = diabetes.data
y = diabetes.target

# 훈련 데이터와 테스트 데이터로 분리 (테스트 사이즈 30%)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=42) # 재현성을 위해 random_state 추가

# 라쏘(Lasso) 회귀 모델 생성 및 학습
lasso = linear_model.Lasso()
model = lasso.fit(train_X, train_y)

# 학습된 모델 객체에서 모든 매개변수를 가져옵니다.
params = model.get_params()

# 매개변수 딕셔너리를 출력합니다.
print(model)
print(params)

# 테스트 데이터에 대한 예측 수행
pred = model.predict(test_X)

# 예측 결과의 일부를 확인
print("\n첫 10개 예측 결과:")
print(pred[:10])

# 1) 평균 제곱 오차 (Mean Squared Error, MSE) 계산
mse = mean_squared_error(test_y, pred)
print(f"\n1) MSE: {mse}")

# 2) 평균 제곱근 오차 (Root Mean Squared Error, RMSE) 계산
rmse = math.sqrt(mse)
print(f"   RMSE: {rmse}")

# 3) 평균 절대 오차 (Mean Absolute Error, MAE) 계산
mae = mean_absolute_error(test_y, pred)
print(f"2) MAE: {mae}")

# 4) 설명 분산 점수 (Explained Variance Score) 계산
evs = explained_variance_score(test_y, pred)
print(f"3) Explained Variance Score: {evs}")

# 5) 결정 계수 (R-squared) 계산
r2 = r2_score(test_y, pred)
print(f"4) R-squared: {r2}")

Lasso()
{'alpha': 1.0, 'copy_X': True, 'fit_intercept': True, 'max_iter': 1000, 'positive': False, 'precompute': False, 'random_state': None, 'selection': 'cyclic', 'tol': 0.0001, 'warm_start': False}

첫 10개 예측 결과:
[155.13699363 165.12252583 166.78532084 199.25042232 142.04184558
 135.60276066 235.41212229 188.48001545 114.01136186 151.76362523]

1) MSE: 3444.670824977634
   RMSE: 58.69131813971836
2) MAE: 49.20790052364458
3) Explained Variance Score: 0.36190018091988974
4) R-squared: 0.36189802861957876


##### 코드 해설

##### 코드 해설

## 핵심 요약 (Key Takeaways)

  - **하나의 지표만 맹신하지 마세요**: 비즈니스 문제에 따라 적합한 평가 지표가 다름. 평균적인 오차 크기가 중요하다면 **MAE**, 큰 실수를 피해야 한다면 \*\*RMSE(MSE)\*\*를 사용하는 등 상황에 맞는 종합적인 평가가 필요함.
  - **안정적인 성능 평가를 원한다면 교차 검증은 필수입니다**: 데이터 분할에 따른 우연성을 줄이고 모델의 **일반화 성능**을 더 정확하게 측정하기 위해 **K-폴드 교차 검증**을 습관화하는 것이 좋음.
  - **`Scikit-learn`과 `Statsmodels`는 용도가 다릅니다**: \*\*`Scikit-learn`\*\*은 **예측 모델링과 성능 평가**에 최적화되어 있고, \*\*`Statsmodels`\*\*는 변수 간의 **통계적 관계를 깊이 있게 분석**하고 해석하는 데 더 강력함.